Target: implement below SQL code by RDDs
```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(tolls_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

---

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 01:12:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.take(5)

[Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2021, 6, 18, 11, 31), lpep_dropoff_datetime=datetime.datetime(2021, 6, 18, 11, 37), store_and_fwd_flag=None, RatecodeID=None, PULocationID=210, DOLocationID=155, passenger_count=None, trip_distance=1.06, fare_amount=16.3, extra=2.75, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=19.35, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2021, 6, 14, 15, 54, 46), lpep_dropoff_datetime=datetime.datetime(2021, 6, 14, 16, 5, 31), store_and_fwd_flag='N', RatecodeID=1, PULocationID=65, DOLocationID=189, passenger_count=1, trip_distance=1.3, fare_amount=8.5, extra=1.0, mta_tax=0.5, tip_amount=2.55, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=12.85, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2021, 6, 11, 16, 28), 

select

rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [8]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2021, 6, 18, 11, 31), PULocationID=210, total_amount=19.35),
 Row(lpep_pickup_datetime=datetime.datetime(2021, 6, 14, 15, 54, 46), PULocationID=65, total_amount=12.85),
 Row(lpep_pickup_datetime=datetime.datetime(2021, 6, 11, 16, 28), PULocationID=75, total_amount=27.36),
 Row(lpep_pickup_datetime=datetime.datetime(2021, 6, 12, 9, 29, 47), PULocationID=95, total_amount=49.6),
 Row(lpep_pickup_datetime=datetime.datetime(2021, 6, 1, 14, 32, 17), PULocationID=74, total_amount=22.56)]

filter (vs WHERE)

In [9]:
from datetime import datetime

In [10]:
start = datetime(year=2020, month=1, day=1)

1) lambda

In [11]:
rdd.filter(lambda row: row.lpep_pickup_datetime >= start).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2021, 6, 18, 11, 31), PULocationID=210, total_amount=19.35)]

2) function

In [12]:
def filter_outliner(row):
    return row.lpep_pickup_datetime >= start

In [13]:
rdd.filter(filter_outliner).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2021, 6, 18, 11, 31), PULocationID=210, total_amount=19.35)]

map (vs GROUP BY)

In [18]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)

In [19]:
rdd \
    .filter(filter_outliner) \
    .map(prepare_for_grouping) \
    .take(10)

[((datetime.datetime(2021, 6, 18, 11, 0), 210), (19.35, 1)),
 ((datetime.datetime(2021, 6, 14, 15, 0), 65), (12.85, 1)),
 ((datetime.datetime(2021, 6, 11, 16, 0), 75), (27.36, 1)),
 ((datetime.datetime(2021, 6, 12, 9, 0), 95), (49.6, 1)),
 ((datetime.datetime(2021, 6, 1, 14, 0), 74), (22.56, 1)),
 ((datetime.datetime(2021, 6, 25, 16, 0), 41), (10.3, 1)),
 ((datetime.datetime(2021, 6, 21, 9, 0), 56), (24.5, 1)),
 ((datetime.datetime(2021, 6, 23, 18, 0), 65), (11.3, 1)),
 ((datetime.datetime(2021, 6, 13, 16, 0), 75), (5.3, 1)),
 ((datetime.datetime(2021, 6, 6, 13, 0), 89), (56.75, 1))]

reduce by key

In [21]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [23]:
rdd \
    .filter(filter_outliner) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)

[((datetime.datetime(2021, 6, 1, 14, 0), 74), (372.70000000000005, 16)),
 ((datetime.datetime(2021, 6, 24, 14, 0), 215), (18.5, 1)),
 ((datetime.datetime(2021, 6, 11, 17, 0), 181), (93.47999999999999, 5)),
 ((datetime.datetime(2021, 6, 1, 7, 0), 65), (50.35, 1)),
 ((datetime.datetime(2021, 6, 1, 12, 0), 17), (10.8, 1)),
 ((datetime.datetime(2021, 6, 7, 23, 0), 42), (47.55, 3)),
 ((datetime.datetime(2021, 6, 16, 16, 0), 75), (547.0600000000002, 26)),
 ((datetime.datetime(2021, 6, 17, 17, 0), 74), (354.6400000000001, 21)),
 ((datetime.datetime(2021, 6, 23, 15, 0), 129), (97.14, 3)),
 ((datetime.datetime(2021, 6, 11, 12, 0), 208), (33.55, 1))]

unwrap

In [28]:
from collections import namedtuple

In [29]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [32]:
def unwrap(row):
    return RevenueRow(hour=row[0][0], 
            zone=row[0][1], 
            revenue=row[1][0], 
            count=row[1][1])

In [33]:
rdd \
    .filter(filter_outliner) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF() \
    .show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2021-06-01 14:00:00|  74|372.70000000000005|   16|
|2021-06-24 14:00:00| 215|              18.5|    1|
|2021-06-11 17:00:00| 181| 93.47999999999999|    5|
|2021-06-01 07:00:00|  65|             50.35|    1|
|2021-06-01 12:00:00|  17|              10.8|    1|
|2021-06-07 23:00:00|  42|             47.55|    3|
|2021-06-16 16:00:00|  75| 547.0600000000002|   26|
|2021-06-17 17:00:00|  74| 354.6400000000001|   21|
|2021-06-23 15:00:00| 129|             97.14|    3|
|2021-06-11 12:00:00| 208|             33.55|    1|
|2021-06-29 15:00:00|  75|496.18000000000006|   25|
|2021-06-05 14:00:00|  74|377.15000000000015|   24|
|2021-06-10 15:00:00|  18|              11.3|    1|
|2021-06-17 14:00:00|  98|             30.11|    1|
|2021-06-03 17:00:00|  75|423.30000000000007|   24|
|2021-06-07 00:00:00|  42|              21.3|    1|
|2021-06-28 

fix schema

In [34]:
df_result = rdd \
    .filter(filter_outliner) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF()

In [35]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', LongType(), True), StructField('revenue', DoubleType(), True), StructField('count', LongType(), True)])

In [36]:
from pyspark.sql import types

In [37]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [38]:
df_result = rdd \
    .filter(filter_outliner) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [39]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2021-06-01 14:00:00|  74|372.70000000000005|   16|
|2021-06-24 14:00:00| 215|              18.5|    1|
|2021-06-11 17:00:00| 181| 93.47999999999999|    5|
|2021-06-01 07:00:00|  65|             50.35|    1|
|2021-06-01 12:00:00|  17|              10.8|    1|
|2021-06-07 23:00:00|  42|             47.55|    3|
|2021-06-16 16:00:00|  75| 547.0600000000002|   26|
|2021-06-17 17:00:00|  74| 354.6400000000001|   21|
|2021-06-23 15:00:00| 129|             97.14|    3|
|2021-06-11 12:00:00| 208|             33.55|    1|
|2021-06-29 15:00:00|  75|496.18000000000006|   25|
|2021-06-05 14:00:00|  74|377.15000000000015|   24|
|2021-06-10 15:00:00|  18|              11.3|    1|
|2021-06-17 14:00:00|  98|             30.11|    1|
|2021-06-03 17:00:00|  75|423.30000000000007|   24|
|2021-06-07 00:00:00|  42|              21.3|    1|
|2021-06-28 

In [40]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', IntegerType(), True), StructField('revenue', DoubleType(), True), StructField('count', IntegerType(), True)])

In [41]:
df_result.write.parquet('tmp/green-revenue')